In [1]:
import os
os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [4]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions import split_train_validation_random_holdout


URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)



result,topK,l1_ratio,alpha

0.06001721416196575,142.0,0.4033590645217344,9.521340863590419e-05

In [5]:
from Recommenders.SLIM.SLIMElasticNetRecommender_Peppe import MultiThreadSLIM_SLIMElasticNetRecommender

recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train_validation)

topK = int(142.0)
alpha = 9.521340863590419e-05
l1_ratio = 0.4033590645217344
recommender.fit(topK=topK, l1_ratio=l1_ratio, alpha=alpha)

Progress: 100%|██████████| 38121/38121 [08:50<00:00, 71.83items/s]


In [15]:
# Check the 20% of the URM_test
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
recommendations = []
# Get user_id from the test set
user_id = np.unique(URM_test.nonzero()[0])
print(user_id)

for id in user_id:
    # Get the recommendations
    recommendations.append(recommender.recommend(id, cutoff=10))

submission = pd.DataFrame(columns=['user_id', 'item_list'])
print(user_id)
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

print(submission.head(10))
# Save the submission file
submission.to_csv("submissions/slimnet_ERNESTO.csv", index=False)

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
[    0     1     2 ... 35733 35734 35735]
[    0     1     2 ... 35733 35734 35735]
   user_id                                          item_list
0        0     3653 471 8604 3074 1425 6348 116 2743 399 7548
1        1  11699 3130 13778 13733 15088 9812 13766 2644 1...
2        2  29656 22471 15542 31131 4851 18518 1195 21495 ...
3        3  25643 25079 11753 6827 9742 7448 27392 9115 16...
4        4  3422 11753 9812 17624 15694 15902 527 3472 152...
5        5  28805 4280 39 4253 2 15648 13005 19209 19208 2782
6        6  4636 8505 9747 18609 3532 15139 8661 125 6822 ...
7        7  5436 15137 3077 2843 515 2892 3767 3070 15104 ...
8        8  627 19497 2566 21162 211 7780 2837 3844 7778 1...
9        9  11753 12600 4142 446 24304 2565 7034 22497 399...
